## Data load

In [76]:
import json
from openai import OpenAI
from key_info import OPENAI_API_KEY, UPSTAGE_API_KEY

chat_model = 'gpt-3.5-turbo' # gpt-4-0125-preview, gpt-4-1106-preview, gpt-3.5-turbo
client = OpenAI(api_key = OPENAI_API_KEY)

In [77]:
with open("./result/KMI_정신건강_id1~.json", "r") as json_file:
    data = json.load(json_file)
len(data)

193

In [78]:
def from_gpt(prompt):    
    prompt = '''
    너는 동기강화학습의 태그 분류기야. 
    내가 앞으로 말해줄 정의들을 보고 마지막 내담자의 발화가 어떤 정의에 해당하는지 태그 이니셜을 이용해서 대답해줘.
    
    [SR] : 단순 반영하기로, 내담자가 한 말에 정보를 더 추가하기보다는 내담자가 한 말에 가장 가깝게 반영하는 것을 말한다.
    [CR] : 복합 반영하기로, 상담의 내용이나 내담자의 생각, 감정을 강조하는 재진술이나 긍정적인 변화에 나아갈 수 있게 하기 위해 내담자의 말에 의미를 덧붙인 상담사의 재진술을 의미한다.
    [OQ] : 열린 질문하기로, 열린 질문은 특정한 방향에 초점을 두고 내담자가 자유롭게 생각하고 구체적으로 답할 수 있도록 하는 질문이다.
    [CQ] : 닫힌 질문하기로, 상담사가 내담자에게 한 질문에 내담자가 예/아니오 혹은 단답식의 응답을 하게 하는 질문 형식을 말한다.
    [AF] : 인정하기로, 내담자의 특별한 강점과 능력, 좋은 의도, 시도한 노력들에 대해서 알아차리고 언급해주는 것이다. 내담자의 긍정적인 부분을 강조하는 기술이다.
    [GI] : 정보 제공하기로, 내담자를 설득하려 하거나 자신의 개인적인 의견이나 관점을 덧붙이지 않고 일반적이고 객관적인 정보를 내담자에게 제공한다.
    [AD] : 조언하기로, 내담자에게 조언이나 제안을 하거나, 해결책이나 대안을 제시하거나, 내담자가 할 수 있는 행동 등을 알려주어 내담자의 변화를 설득하려는 진술이다.
    [OT] : 그 외의 태그로, 앞의 조건에 해당하지 않는 나머지 발화를 의미한다.    
    
    '''+ prompt + ' \n상담사 발화의 태그 :'
    response = client.chat.completions.create(
        model=chat_model,
        messages=[{"role": "user", "content": prompt}],
        stream=False,
    ).choices[0].message.content
    return response


def from_gpt_fewshot(prompt):    
    prompt = '''
    너는 동기강화학습의 태그 분류기야. 
    내가 앞으로 말해줄 정의들을 보고 마지막 내담자의 발화가 어떤 정의에 해당하는지 태그 이니셜을 이용해서 대답해줘.
    
    [SR] : 단순 반영하기로, 내담자가 한 말에 정보를 더 추가하기보다는 내담자가 한 말에 가장 가깝게 반영하는 것을 말한다.
    [CR] : 복합 반영하기로, 상담의 내용이나 내담자의 생각, 감정을 강조하는 재진술이나 긍정적인 변화에 나아갈 수 있게 하기 위해 내담자의 말에 의미를 덧붙인 상담사의 재진술을 의미한다.
    [OQ] : 열린 질문하기로, 열린 질문은 특정한 방향에 초점을 두고 내담자가 자유롭게 생각하고 구체적으로 답할 수 있도록 하는 질문이다.
    [CQ] : 닫힌 질문하기로, 상담사가 내담자에게 한 질문에 내담자가 예/아니오 혹은 단답식의 응답을 하게 하는 질문 형식을 말한다.
    [AF] : 인정하기로, 내담자의 특별한 강점과 능력, 좋은 의도, 시도한 노력들에 대해서 알아차리고 언급해주는 것이다. 내담자의 긍정적인 부분을 강조하는 기술이다.
    [GI] : 정보 제공하기로, 내담자를 설득하려 하거나 자신의 개인적인 의견이나 관점을 덧붙이지 않고 일반적이고 객관적인 정보를 내담자에게 제공한다.
    [AD] : 조언하기로, 내담자에게 조언이나 제안을 하거나, 해결책이나 대안을 제시하거나, 내담자가 할 수 있는 행동 등을 알려주어 내담자의 변화를 설득하려는 진술이다.
    [OT] : 그 외의 태그로, 앞의 조건에 해당하지 않는 나머지 발화를 의미한다.  
    
    
    내담자: 제 여동생처럼만 되었으면 좋겠어요.
    상담사: 동생처럼 되고 싶으시군요.  
    상담사 발화의 태그 : [SR]
    
    내담자: 마누라 때문에 짜증이 납니다.
    상담사: 엄청 화가 많이 나셨군요.
    상담사 발화의 태그 : [CR]
    
    내담자: 제게 문제가 있다는 것을 인정해요. 하지만 약물남용 치료가 필요하지는 않아요.
    상담사: 그렇다면 무엇이 내담자님에게 도움이 될까요?
    상담사 발화의 태그 : [OQ]
    
    내담자: 진로가 고민돼요
    상담사: 내담자님은 장래를 위해 진학이나 취업 중에 어떤 것을 선택할 건가요?
    상담사 발화의 태그 : [CQ]
    
    내담자: 가족과 대화를 늘리려고 어제 거실에서 가족들과 함께 TV를 봤어요.
    상담사: 내담자님이 할 수 있는 최선을 다하셨네요. 정말 잘하셨습니다.
    상담사 발화의 태그 : [AF]
    
    내담자: C형 간염이 있지만, 술이 너무 먹고싶어요
    상담사: C형 간염이 있는 사람이 음주를 지속할 경우, 보통 사람에 비해 간에 더 많은 손상을 줄 수 있습니다.'
    상담사 발화의 태그 : [GI]
    
    내담자: 학교에서의 일이 너무 스트레스에요
    상담사: 부모님께 먼저 말씀을 드려보는 건 어떨까요?
    상담사 발화의 태그 : [AD]
    
    내담자: 담배를 계속 피우고 싶어서 불가능해요.
    상담사: 그렇죠. 밥은 먹어야 하고 운전도 해야 하니까요.
    상담사 발화의 태그 : [OT]  
    
    
    
    '''+ prompt + ' \n상담사 발화의 태그 :'
    response = client.chat.completions.create(
        model=chat_model,
        messages=[{"role": "user", "content": prompt}],
        stream=False,
    ).choices[0].message.content
    return response

In [79]:
tag_dict = {'[SR]' : 'Simple Reflection',
            '[CR]' : 'Complex Reflection',
            '[OQ]' : 'Open Question',
            '[CQ]' : 'Closed Question',
            '[AF]' : 'Affirm',
            '[GI]' : 'Give Information',
            '[AD]' : 'Advise',
            '[OT]' : 'General'
            }

## Real 'General' tag filtering

In [80]:
with open("./result/KMI_정신건강_id1~.json", "r") as json_file:
    data = json.load(json_file)
len(data)

idx = 0
prev_utterance = ''
for i in data:
    for j in i['dialogue']:
        if j['label'] == 'General':
            utterance = j['utterance_ko']
            
            talk = '내담자: '+prev_utterance+'\n'+'상담사: '+utterance 
            print(talk)
            predicted_tag = from_gpt_fewshot(talk)
            # predicted_tag = from_gpt(talk)            
            predicted_tag = predicted_tag.strip()
            if predicted_tag not in tag_dict.keys():
                predicted_tag = '[OT]'
            predicted_tag = tag_dict[predicted_tag]
            print(predicted_tag)
            j['label'] = predicted_tag
        prev_utterance = j['utterance_ko']
    idx += 1
    if idx == 100:
        break

내담자: 다른 사람들이 열정적으로 살아가는 것을 보며 저도 저렇게 해야 한다고 생각하지만, 의욕이 들지 않아요.
상담사: 타인의 열정적인 모습을 보며 무엇을 해야 할지 고민하셨군요.
Simple Reflection
내담자: 네, 그 감정들 때문에 제가 본래 가지고 있던 열정도 잃어버린 것 같아요.
상담사: 원래 가지고 계셨던 열정을 잃어버렸다고 느끼시는군요.
Simple Reflection
내담자: 네, 그리고 저 열정을 되찾고 싶어요.
상담사: 열정을 되찾고 싶어하는 마음이 드러나고 있군요.
Simple Reflection
내담자: 네, 그 의지를 가지고 더 좋은 방향으로 나아가고 싶어요.
상담사: 더 좋은 방향으로 나아가고자 하는 의지가 강해 보이네요.
Affirm
내담자: 안녕하세요. 제가 너무 많은 일들을 겪었던 탓에, 어느새 자신감이 많이 떨어져 버렸습니다.
상담사: 그런 경험들이 자신에 대한 믿음을 잃게 만들었군요.
Complex Reflection
내담자: 제 마음을 조금씩 열고 다른 사람들과의 소통을 늘리고 싶습니다.
상담사: 소통을 통해 마음을 열고자 하는 의지가 이 어려움을 극복하는 데 큰 역할을 할 것입니다.
Affirm
내담자: 이제부터라도 자신에 대해 조금씩 긍정적으로 생각하며 이전의 저를 되찾고 싶습니다.
상담사: 변화를 향한 내담자님의 의지가 새로운 길을 열어갈 것입니다.
Affirm
내담자: 매일 아침 일어나서 새로운 하루를 시작해야 한다는 걸 생각하면 숨이 막혀요.
상담사: 그러신가요. 하루의 시작이 무겁게 느껴지시는군요.
Complex Reflection
내담자: 그렇습니다. 제 상황에 대해 앞으로 어떻게 나아가야 할지 조금이라도 가이드를 얻고 싶어요.
상담사: 내담자님의 현재 상황에서 나아가기 위한 방향을 찾고 싶으시군요.
Complex Reflection
내담자: 예, 제 삶에 조금이라도 긍정적인 변화를 주고 싶습니다.
상담사: 현재 상황에서 긍정적인 변화를 추구하시는 마음이 강하시군요.
Affirm
내담자: 네, 저는 제 

In [20]:
data

[{'id': 1,
  'category': '정신건강',
  'context': '다른 사람들이 열정적인게 마음이 불안하고 불편하고 그래요. 그걸 보면 나도 저렇게 해야는 하는데… 싶지만 의욕은 하나도 없고 왜 다른 사람들은 저렇게 열심히 사는데 나는 안그러지? 에 대한 죄책감도 들고 열심히 살지 않는 나에 대한 자책만 늘고 다들 어떻게 저렇게 열정적이지? 하면서 나도 저럴때가 있었는데.. 하는 마음에 그 열정이 부럽고 어떤 동기여부가 있는건지 궁금하기도 해요 저는 다른 사람들 앞에 서야하는 직업을 갖고싶고 그에 맞는 학원을 다니고 있는데 이제 남들앞에 서는게 무서워요. 공황 같은건 아닌거 같은데 .. 그냥 다른 사람들이 나보다 잘 할것 같고 내가 하는 거 보고 비웃을 거 같고 별 볼일 없다 생각할 것 같고 .. 그런 생각들 때문에 완벽하지 못한 모습을 보여주는 것이 두려워요 그래서 완벽 이라는 강박도 생긴것 같아요 틀리기 싫고 틀릴까봐 겁나고 피드백이 겁나요 내가 준비한 것이 틀렸다고 하면 내 세상이 무너지는 것 같고 갈피를 못잡겠어요 완벽을 추구하기위해 여러시도를 해보는데 기한은 정해져있으니 이거저거 시도해 보다 시간이 다 가서 결국 제게 남은 결과는 이도저도 아닌 쓰레기가 남아요 그래서 이걸 여러사람앞에서 보여주기가 겁나고 자신감도 자존감도 없어요 예전의 나는 이렇지 않았는데 하면서 과거만 추억 할 뿐이에요 현실을 살아야하는데.. 그냥 너무 답답해요 자꾸 의욕도 안나고 학원 가는 것도 겁나고 하니까 내가 이걸 좋아해서 시작한게 맞는지 내 길이 아닌거 같고 하면서 답답한 마음에.. 주절주절 쓰게 됐네요 …',
  'turns': 15,
  'dialogue': [{'role': 'Therapist',
    'utterance_ko': '안녕하세요, 최근에 어떤 생각이나 감정이 반복되었나요?',
    'utterance_en': 'Hi, what thoughts or feelings have you been having recently that keep